**Pre-processing done**

**Transfer learning begin:**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from collections import Counter                     #Written by Jacob and used to calculate the true accuracy.
import numpy as np


# actual accuracy
def get_accuracy(p_data, t_data, extra_data):
    sc = 0
    if extra_data is None:
        for i in range(len(p_data)):
            if p_data[i] == t_data[i]:
                sc += 1

        return sc / len(p_data)
    else:
        counter = -1
        for i in range(len(extra_data)):
            is_good = True
            for j in range(extra_data[i]):
                counter += 1
                is_good &= p_data[counter] == t_data[counter]
            if is_good:
                sc += 1
        return sc / len(extra_data)

In [4]:
import os

# third-party library
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
  
        
DOWNLOAD_MNIST = False

In [5]:
torch.manual_seed(1)
# load the feature network
feature_net = torch.load('/content/drive/MyDrive/fnet.pt')    
# create 3 random images of size 1x28x28
images = torch.rand(3, 1, 28, 28)
# compute the features for the 3 random images
feature_net(images).shape # this shape should be 3x84  
#Seems like we use a CNN to extract the features of the Images. Sounds like the purpose of it is like PCA or other similar methods.

torch.Size([3, 84])

In [6]:
from torchvision.datasets import MNIST

mnist_tr = MNIST('~/.torchvision', train=True, download=True)
x_train, y_train = mnist_tr.data, mnist_tr.targets
mnist_ts = MNIST('~/.torchvision', train=False, download=True)
x_test, y_test = mnist_ts.data, mnist_ts.targets


x_train = x_train.reshape(x_train.shape[0], 1,28, 28)  #Reshape the image to input into the feature net successfully
x_test = x_test.reshape(x_test.shape[0],1,28, 28)
x_train = x_train/255
x_test = x_test/255
x_train = feature_net(x_train)
x_test = feature_net(x_test)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
x_train1nb, y_train1nb = mnist_tr.data, mnist_tr.targets    #Reload the MNIST dataset as the x_train... and pre-process.
x_train1nb = x_train1nb.reshape(len(x_train1nb), -1).float()
Xx= x_train1nb.detach().numpy()
Yy = y_train1nb.detach().numpy()
x_test1nb, y_test1nb = mnist_ts.data, mnist_ts.targets
x_test1nb = x_test1nb.reshape(len(x_test1nb), -1).float()
Xxt= x_test1nb.detach().numpy()
Yyt = y_test1nb.detach().numpy()

torch.Size([60000, 84]) torch.Size([60000])
torch.Size([10000, 84]) torch.Size([10000])


In [7]:
import sklearn
X = x_train.detach().numpy()                 #Pre-process the input to apply NB classifier.
Y = y_train.detach().numpy()
Xt = x_test.detach().numpy()
Yt = y_test.detach().numpy()

In [8]:
import h5py                                  #Load pre-processed data
# Open the file as readonly
h5f = h5py.File('/content/drive/MyDrive/Resizedata.h5', 'r')

# Load the training, test and validation set
x_train = h5f['x_train'][:]
y_train = h5f['y_train'][:]
x_test = h5f['x_test'][:]
y_test = h5f['y_test'][:]

# Close this file
h5f.close()

print('Training set', x_train.shape, y_train.shape)
print('Test set', x_test.shape, y_test.shape)

Training set (73257, 28, 28) (73257,)
Test set (26032, 28, 28) (26032,)


In [9]:
file = open("/content/drive/MyDrive/cf/test_labels.txt", "r")
stringList = file.readlines()
extra_data = []
for i in range(len(stringList)):
    extra_data.append(len(stringList[i])-1)

In [10]:
K = [5,50,100,500,1000,5000,10000]
one_accuracy_nf = []
true_accuracy_nf = []
one_accuracy_wf = []
true_accuracy_wf = []

In [11]:
from sklearn.neighbors import KNeighborsClassifier 
for k in K:
  knn1 = KNeighborsClassifier(n_neighbors=k)
  knn1.fit(X, Y)
  knn2 = KNeighborsClassifier(n_neighbors=k)
  knn2.fit(Xx, Yy)
  x_test2nb = x_test.reshape(len(x_test), -1)
  one_accuracy_nf.append(knn2.score(x_test2nb,y_test))
  x_test2nbe = feature_net(torch.Tensor(x_test.reshape(len(x_test),1,28,28)/255))
  x_test2nbe = x_test2nbe.detach().numpy()
  one_accuracy_wf.append(knn1.score(x_test2nbe,y_test))
  p = knn2.predict(x_test2nb)
  true_accuracy_nf.append(get_accuracy(p,y_test,extra_data))
  p2 = knn1.predict(x_test2nbe)
  true_accuracy_wf.append(get_accuracy(p2,y_test,extra_data))

In [17]:
print(one_accuracy_nf)

[0.18189151813153043, 0.17835740626920712, 0.17624462200368776, 0.1672940995697603, 0.16176244622003688, 0.14678088506453596, 0.14163337430854334]


In [14]:
print(one_accuracy_wf)

[0.2258758451137062, 0.19403042409342347, 0.18761524277811925, 0.19875537799631224, 0.20167486170866625, 0.18365857406269206, 0.18185310387215733]


In [15]:
print(true_accuracy_nf)

[0.07001836547291093, 0.06818181818181818, 0.06512090602999694, 0.05777471686562596, 0.05218855218855219, 0.03948576675849403, 0.03558310376492195]


In [16]:
print(true_accuracy_wf)

[0.067722681359045, 0.0687174778083869, 0.0660391796755433, 0.06932966023875115, 0.06802877257422713, 0.05501989592898684, 0.05371900826446281]


In [13]:
one_accuracy_nf
one_accuracy_wf
true_accuracy_nf
true_accuracy_wf

[0.067722681359045,
 0.0687174778083869,
 0.0660391796755433,
 0.06932966023875115,
 0.06802877257422713,
 0.05501989592898684,
 0.05371900826446281]